In [1]:
#import packages
import pandas as pd
import requests
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request, json
import requests
import urllib.parse
import datetime
import nltk
nltk.download([
        "names",
        "stopwords",
        "averaged_perceptron_tagger",
        "vader_lexicon",
        "punkt",
        'wordnet'])
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re, string

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package names to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def get_data(url, uniqueid, startdatetime, enddatetime, cat_clnm, subcat_clnm, subsubcat_clnm, datetime_clnm):
    '''
    Inputs: 
    url - string value of SODA API url
    startdatetime & enddatetime - string values of the format 'YYYY-MM-DDTHH:MM:SS', i.e. '2019-12-31T23:59:59.000' 
                                for the beginning and end of timeframe of interest
    datetime_clnm - string value of the column name in the data for the datetime of complaint inquiry
    
    Output:
    gdf - GeoDataFrame of 311 inquiries. Geometry is point location of 311 inquiry.
    '''
    query = (url +'?'
            "$select= count(*) as count,%s as cat, %s as subcat, %s as subsubcat"
            "&$where=%s between '"+startdatetime+"' and '"+enddatetime+"'"
            "&$group=cat,subcat,subsubcat"
            "&$order=count DESC"
            "&$limit=500000"
            )%(cat_clnm,subcat_clnm,subsubcat_clnm,datetime_clnm)
    query = query.replace(" ", "%20")
    print(query)
    response = urllib.request.urlopen(query)
    data = json.loads(response.read())
    
    return data

In [3]:
all311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
uniqueid = 'unique_key'
cat_clnm = 'agency_name'
subcat_clnm = 'complaint_type'
subsubcat_clnm = 'descriptor'
datetime_clnm = 'created_date'

data = get_data(all311, uniqueid, '2019-01-01T00:00:00.000', '2020-01-01T00:00:00.000', cat_clnm, subcat_clnm, subsubcat_clnm, datetime_clnm)

https://data.cityofnewyork.us/resource/erm2-nwe9.json?$select=%20count(*)%20as%20count,agency_name%20as%20cat,%20complaint_type%20as%20subcat,%20descriptor%20as%20subsubcat&$where=created_date%20between%20'2019-01-01T00:00:00.000'%20and%20'2020-01-01T00:00:00.000'&$group=cat,subcat,subsubcat&$order=count%20DESC&$limit=500000


In [4]:
all_df = pd.DataFrame(data, columns = data[0].keys())
all_df.head()

,count,cat,subcat,subsubcat
0,214808,Department of Sanitation,Request Large Bulky Item Collection,Request Large Bulky Item Collection
1,147964,New York City Police Department,Noise - Residential,Loud Music/Party
2,137887,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING
3,98815,New York City Police Department,Blocked Driveway,No Access
4,77815,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party


In [5]:
calls = pd.read_csv('../Outputs/311calls_complaints_vadar_results.csv', index_col = 0)
calls.head()

,count,cat,subcat,subsubcat,combined_cats,combined_cats_lower,processed_comps,sentiment_scores,neg,neu,pos,compound
0,84545,Department of Finance,Parking Ticket Lookup - Ticket or Plate Number...,Status of a parking ticket. Must know ticket o...,Department of Finance Parking Ticket Lookup - ...,department of finance parking ticket lookup - ...,department finance parking vehicles ticket loo...,"{'neg': 0.0, 'neu': 0.874, 'pos': 0.126, 'comp...",0.000,0.874,0.126,0.1531
1,76516,3-1-1 Call Center,Service Request Status - General,Status of most Service Requests filed with 311.,3-1-1 Call Center Service Request Status - Gen...,3-1-1 call center service request status - gen...,call center service request status general sta...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
2,60523,Department of Finance,Property Tax Account Assistance and Bill Infor...,Help with questions about the property tax bil...,Department of Finance Property Tax Account Ass...,department of finance property tax account ass...,department finance property building tax accou...,"{'neg': 0.0, 'neu': 0.876, 'pos': 0.124, 'comp...",0.000,0.876,0.124,0.4019
3,57674,NaN,Bulk Item Disposal,"Learn how to get rid of bulk items. Schedule, ...",Bulk Item Disposal Learn how to get rid of bu...,bulk item disposal learn how to get rid of bu...,bulk item disposal learn get rid bulk items sc...,"{'neg': 0.111, 'neu': 0.889, 'pos': 0.0, 'comp...",0.111,0.889,0.000,-0.2500
4,57213,NaN,Heat or Hot Water Complaint in a Residential B...,Report a problem with heat or hot water in a r...,Heat or Hot Water Complaint in a Residential ...,heat or hot water complaint in a residential ...,heat hot water complaint residential building ...,"{'neg': 0.308, 'neu': 0.692, 'pos': 0.0, 'comp...",0.308,0.692,0.000,-0.5994


In [6]:
len(calls.subcat.unique())

5424

In [7]:
len(all_df.subcat.unique())

370

In [8]:
all_df['subcat'] = all_df['subcat'].str.strip().str.lower()
all_df['subcat'] = all_df['subcat'].replace('fatf','financial action task force')
all_df['subcat'] = all_df['subcat'].replace('fcst','weather forcast')
all_df['subcat'] = all_df['subcat'].replace('atf','bureau of alcohol, tabacco, and firearms')

complaints = all_df['subcat'].str.lower().unique().tolist()

In [9]:
calls['subcat'] = calls['subcat'].str.strip().str.lower()
calls['subcat'] = calls['subcat'].replace('fatf','financial action task force')
calls['subcat'] = calls['subcat'].replace('fcst','weather forcast')
calls['subcat'] = calls['subcat'].replace('atf','bureau of alcohol, tabacco, and firearms')

In [10]:
raw_complaints = calls['subcat'].unique().tolist()
print(len(raw_complaints))

5423


In [11]:
remove =['research questions', 'msother', '../../../web-inf/web.xml;x=', '%c0%ae/%c0%ae/web-inf/web.xml',
         '../../web-inf/web.xml;x=', '.../....///.../....///.../....', '%2fetc%2fpasswd', '../../web-inf/web.xml',
         '%c0%ae/%c0%ae/%c0%ae/%c0%ae...', '%e5%98%8a%e5%98%8dx-injecti...', '../web-inf/web.xml;x=',
         '%c0%ae/%c0%ae/%c0%ae/web-in...', '%c0%ae/web-inf/web.xml', '.../...//.../...//.../...//...',
         'bcd9&lt;!--esi--&gt;1ggp&lt...', '%{(#dm=@ognl.ognlcontext@de...', '../web-inf/web.xml',
         '../../../web-inf/web.xml', '../../../../web-inf/web.xml', "{!xmlparser v='&lt;!doctype...",
         'file:///etc/passwd', "a'a\\'b&quot;c&gt;?&gt;%}}%%...", '@(9313*3464)', "kawn''lcgv", '1430*4963',
         '() { _; } &gt;_', '${3804*3137}', '() { :;}; /bin/sleep 0', "misc. comments'+(function()...",
         '`sleep 11`', "&quot;--&gt;'--&gt;`--&gt;&...", '%{4761*8506}', '() { :;}; /bin/sleep 11', '$(sleep 11)',
         'misc. comments&gt;  bcc:bl0...', 'misc. comments  bcc:a5glxzf...', '*)(!(objectclass=*)',
         'idexf3mrb7)(!(objectclass=*)', '*)(objectclass=*', '&lt;vlv xmlns=&quot;http://...',
         '&lt;odb xmlns:xi=&quot;http...', 'misc. comments]]&gt;&gt;&lt;', 'g0oii1s4s5)(objectclass=*',
         'misc. comments../../../../....', '/./validate_form.php', './validate_form.php',
         '../../../../../../../../../...', '.../misc. comments', '..\\..\\..\\..\\..\\..\\..\\..\\..\\...',
         '..././..././..././..././......', '%2e%2e%2f%2e%2e%2f%2e%2e%2f...', '/etc/passwd',
         'misc. comments..\\..\\..\\..\\....', 'c:\\windows\\win.ini misc. co...', '%2e%2e%5c%2e%2e%5c%2e%2e%5c...',
         '...\\./...\\./...\\./...\\./......', 'ztestint', 'zsystest', 'select message type...', '\\windows\\win.ini',
         '...\\.\\...\\.\\...\\.\\...\\.\\......', 'file:///c:/windows/win.ini', "misc. comments'|ping -c 21 ...",
         '.../.\\.../.\\.../.\\.../.\\......', 'c:\\windows\\win.ini', "misc. comments'|echo y9v6fc...",
         'misc. comments|ping -c 21 1...', 'misc. comments&amp;ping -n ...', 'misc. comments|echo 3p7wexz...',
         'misc. comments|ping -n 21 1...', 'misc. comments&quot;|ping -...', 'misc. comments&amp;echo xjd...',
         'misc. comments&amp;nslookup...', 'misc. comments&quot;|echo 2...', "misc. comments'&quot;`0&amp...", 
         'misc. comments|nslookup -q=...', "misc. comments'.sleep(20).'", '266dyrgvhg1egsyp2oq57axbm2s...',
         'http://g7orz5h9iu2sh6z332rj...', 'misc. comments{${sleep(20)}}', 'nslookup -q=cname nneyfcxgy...',
         "eval(compile('for x in rang...", 'yw97y0gi2s', "misc. comments'+sleep(20.to...", 'hcvn1{{687*375}}gji51',
         "misc. comments'+eval(compil...", "misc. comments%}dt2pq'/&quo...", "misc. comments}}t2v95'/&quo...",
         'misc. commentso0w3r%&gt;xb6...', 'misc. comments&quot;', 'misc. commentseowljujfh8',
         "misc. comments' and 6049=60...", "misc. comments' and 7702=77...", "misc. comments' and '1618'=...",
         "misc. comments')waitfor del...", 'qfix4${695*589}lixaf', "misc. comments',0)waitfor d...",
         "misc. comments' and '7835'=...", "misc. comments47876391' or ...", "misc. comments' and (select...",
         "misc. comments52038737' or ...", "misc. comments' waitfor del...", "misc. comments88909845' or ...",
         "misc. comments69075347' or ...", "misc. comments'+(select*fro...", 'misc. comments,(select*from...',
         "misc. comments'(select*from...", "misc. comments');declare @q...", "misc. comments'+(select loa...",
         "misc. comments'", "(select load_file('\\\\\\\\615h...", 'misc. comments);declare @q ...',
         "misc. comments';declare @q ...", '(select extractvalue(xmltyp...', "misc. comments'||(select ex...", 
         'misc. comments;declare @q v...']

comp_dict_final = {'noise': ['noise - residential','noise survey', 'noise - commercial', 'noise - helicopter', 'collection truck noise', 'noise - vehicle', 'noise - house of worship', 'noise', 'noise - park', 'noise - street/sidewalk'],
                   'taxi complaints & inquiries': ['green taxi complaint', 'green taxi report', 'taxi compliment', 'dispatched taxi compliment', 'dispatched taxi complaint', 'for hire vehicle complaint', 'taxi complaint', 'taxi licensee complaint', 'taxi report', 'fhv licensee complaint', 'for hire vehicle report'],
                   'homelessness': ['homeless person assistance', 'homeless street condition', 'encampment', 'homeless encampment'],
                   'plaster/paint': ['paint/plaster','peeling paint', 'paint - plaster'],
                   'street/sidewalk/road condition': ['street condition', 'dep street condition', 'street light condition', 'root/sewer/sidewalk condition', 'dep sidewalk condition', 'dep bridge condition', 'curb condition', 'lot condition'],
                   'animal abuse': ['animal abuse', 'animal-abuse'],
                   'dead or damaged tree': ['dead tree', 'damaged tree', 'dead/dying tree'],
                   'waste & bulky item collection': ['institution disposal complaint', 'missed collection', 'missed collection (all materials)', 'residential disposal complaint', 'electronics waste appointment', 'seasonal collection', 'request large bulky item collection', 'request xmas tree collection', 'electronics waste'],
                   'litter basket complaints & inquiries': ['overflowing litter baskets', 'litter basket request', 'litter basket / request', 'litter basket complaint'],
                   'recycling basket complaints': ['recycling basket complaint', 'overflowing recycling baskets'],
                   'literature request': ['literature request', 'oem literature request', 'dor literature request', 'dof literature request', 'hpd literature request'], 
                   'hazardous materials': ['radioactive material', 'hazardous material', 'hazardous materials'],
                   'broken parking meter': ['broken parking meter', 'broken muni meter'],
                   'street sign complaints': ['highway sign - missing', 'highway sign - damaged', 'highway sign - dangling', 'street sign - dangling', 'street sign - missing', 'street sign - damaged'], 
                   'derelict vehicles': ['derelict vehicle', 'derelict vehicles'], 
                   'bus stop shelter complaint': ['bus stop shelter complaint', 'bus stop shelter placement'],
                   'illegal animal - sold/kept': ['illegal animal - sold/kept', 'illegal animal sold', 'illegal animal kept as pet'],
                   'home delivered meal complaint': ['home delivered meal - missed delivery', 'home delivered meal complaint'],
                   "building marshal's office": ["building marshal's office", 'building marshals office'],
                   'street sweeping complaint': ['sweeping/missed', 'sweeping/missed-inadequate', 'sweeping/inadequate'],
                   'fire alarm serive requests': ['fire alarm - new system', 'fire alarm - modification', 'fire alarm - reinspection', 'fire alarm - replacement'],
                   'general inquiry': ['general question', 'comments', 'general'],
                   'dof property complaints & inquiries': ['dof property - owner issue', 'dof property - update account', 'dof property - rpie issue', 'dof property - city rebate', 'dof property - reduction issue', 'dof property - payment issue', 'dof property - property value', 'dof property - request copy', 'dof property - state rebate'], 
                   'public advocate request': ['advocate - other', 'advocate-prop class incorrect', 'advocate-scrie/drie', 'advocate-prop refunds/credits', 'advocate-business tax', 'advocate-co-opcondo abatement', 'advocate-foreclosure', 'advocate - rpie', 'advocate - levy', 'advocate-commercial exemptions', 'advocate-personal exemptions', 'advocate-ubt', 'advocate - lien', 'advocate-property value'],
                   'dof parking requests': ['dof parking - payment issue', 'dof parking - request status', 'dof parking - request copy', 'dof parking - dmv clearance', 'dof parking - address update', 'dof parking - tax exemption'],
                   'public assembly': ['public assembly - temporary', 'public assembly'], 
                   'dhs advantage program inquiries': ['dhs advantage - tenant', 'dhs advantage - third party', 'dhs advantage -landlord/broker'],
                   'construction': ['general construction', 'construction'],
                   'electrical': ['electric', 'electrical'],
                   'unsanitary condition': ['unsanitary condition'],
                   'asbestos': ['asbestos', 'asbestos/garbage nuisance'],
                   'dirty conditions': ['dirty condition', 'dirty conditions'],
                  'snow' : ['snow','snw','snow or ice','snow removal'],
                  'ferry complaints & inquiries':['ferry inquiry','ferry permit','ferry complaint'],
                  'building facade complaints & inquiries':['facade insp safety pgm','facades']}

In [49]:
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer

stemmer = LancasterStemmer()
# Initialize the Porter stemmer
porter=PorterStemmer()
stop_words = set(stopwords.words('english'))

# Create a new dictionary to store the preprocessed keys and values
comp_dict_stem = {}

# Preprocess the keys and values of comp_dict_final
for key, values in comp_dict_final.items():
    # Convert the key to lowercase and remove punctuation
    key_processed = re.sub('/', ' ', key)
    key_processed = key_processed.lower().translate(str.maketrans('', '', string.punctuation))
    # Stem the key using the Porter stemmer
    stemmed_key = porter.stem(key_processed)
    # Create an empty list to store the processed values
    words = stemmed_key.split()
    words_no_stopwords = [word for word in words if word.lower() not in stop_words]
    stemmed_key_no_stopwords = ' '.join(words_no_stopwords)
    processed_values = []
    for value in values:
#         # Replace forward slashes with spaces in the processed value
#         value_processed = value.replace('/', ' ')
#         value_processed = np.char.replace(value, '/', ' ')
        value_processed = re.sub('/', ' ', value)
        value_processed = re.sub(r'-', ' ', value_processed)
        # Convert the value to lowercase and remove punctuation
        value_processed = value_processed.lower().translate(str.maketrans('', '', string.punctuation))
        # Stem the processed value using the Porter stemmer
        stemmed_value = porter.stem(value_processed)
        words = stemmed_value.split()
        words_no_stopwords = [word.replace('noise','nois') for word in words if word.lower() not in stop_words]
        stemmed_value_no_stopwords = ' '.join(words_no_stopwords)
        processed_values.append(stemmed_value_no_stopwords)
    # Add the stemmed key and processed values to the new dictionary
    comp_dict_stem[stemmed_key_no_stopwords] = processed_values

print(comp_dict_stem)

{'nois': ['nois residenti', 'nois survey', 'nois commerci', 'nois helicopt', 'collection truck nois', 'nois vehicl', 'nois house worship', 'nois', 'nois park', 'nois street sidewalk'], 'taxi complaints inquiri': ['green taxi complaint', 'green taxi report', 'taxi compli', 'dispatched taxi compli', 'dispatched taxi complaint', 'hire vehicle complaint', 'taxi complaint', 'taxi licensee complaint', 'taxi report', 'fhv licensee complaint', 'hire vehicle report'], 'homeless': ['homeless person assist', 'homeless street condit', 'encamp', 'homeless encamp'], 'plaster paint': ['paint plast', 'peeling paint', 'paint plast'], 'street sidewalk road condit': ['street condit', 'dep street condit', 'street light condit', 'root sewer sidewalk condit', 'dep sidewalk condit', 'dep bridge condit', 'curb condit', 'lot condit'], 'animal abus': ['animal abus', 'animal abus'], 'dead damaged tre': ['dead tre', 'damaged tre', 'dead dying tre'], 'waste bulky item collect': ['institution disposal complaint', '

In [50]:
for keys in comp_dict_stem.keys():
    if 'complaint' in keys:
        print(keys)
    elif 'service request' in keys:
        print(keys)

taxi complaints inquiri
litter basket complaints inquiri
recycling basket complaint
street sign complaint
bus stop shelter complaint
home delivered meal complaint
street sweeping complaint
dof property complaints inquiri
ferry complaints inquiri
building facade complaints inquiri


In [51]:
# Update the key from "old_key" to "new_key"
comp_dict_stem["taxi"] = comp_dict_stem.pop("taxi complaints inquiri")
comp_dict_stem["ferry"] = comp_dict_stem.pop("ferry complaints inquiri")
comp_dict_stem["facade"] = comp_dict_stem.pop("building facade complaints inquiri")
comp_dict_stem["litter basket"] = comp_dict_stem.pop("litter basket complaints inquiri")
comp_dict_stem["street sign"] = comp_dict_stem.pop("street sign complaint")
comp_dict_stem["recycling basket"] = comp_dict_stem.pop("recycling basket complaint")
comp_dict_stem["bus stop shelter"] = comp_dict_stem.pop("bus stop shelter complaint")
comp_dict_stem["home delivered meal"] = comp_dict_stem.pop("home delivered meal complaint")
comp_dict_stem["street sweeping"] = comp_dict_stem.pop("street sweeping complaint")
comp_dict_stem["dof property"] = comp_dict_stem.pop("dof property complaints inquiri")
comp_dict_stem['fire alarm'] = comp_dict_stem.pop('fire alarm serive request')

In [52]:
lemmatizer = WordNetLemmatizer() 

def stemSentence(sentence):    
    token_words=word_tokenize(sentence)

    stem_sentence=[]
    for word in token_words:       
        if porter.stem(lemmatizer.lemmatize(word)) in ['wast', 'trash', 'garbag', 'refus', 'recycl']:
            stem_sentence.append('wast')
            stem_sentence.append(" ")
            
        elif porter.stem(lemmatizer.lemmatize(word)) in ['bldg', 'bldgmaint']:
            stem_sentence.append('build')
            stem_sentence.append(" ")
            
        elif porter.stem(lemmatizer.lemmatize(word)) in ['vio']:
            stem_sentence.append('violat')
            stem_sentence.append(" ")
            
        else:
            stem_sentence.append(porter.stem(lemmatizer.lemmatize(word)))
            stem_sentence.append(" ")
    
    x = "".join(stem_sentence)
    
    edited_sentence = stem_sentence
    
    return "".join(edited_sentence)

In [53]:
print(len(complaints))
complaints = [complaint for complaint in complaints if complaint not in remove]
print(len(complaints))

247
247


In [20]:
# # loop through each complaint in the list
# for i in range(len(complaints)):
#     complaint = complaints[i]
#     # loop through each key in the dictionary
#     for key in comp_dict_final.keys():
#         # if the complaint is in the list of values for this key
#         if complaint in comp_dict_final[key]:
#             # replace the complaint with the key
#             complaints[i] = key
#             # break out of the inner loop since we found a match
#             break

In [21]:
# complaints = list(set(complaints))
# len(complaints)

In [54]:
#process complaints and raw complaints
complaints_temp = []
for item in complaints:
    item = np.char.replace(item, '/', ' ')
    symbols = "!\"\'#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        item = np.char.replace(item, i, '')
    complaints_temp.append(str(item))

comps_stem = []
for s in range(len(complaints_temp)):
    x = stemSentence(complaints_temp[s])
    comps_stem.append(x)
comps_stem = [elem.strip() for elem in comps_stem]
print(comps_stem[0:25])

['request larg bulki item collect', 'nois residenti', 'heat hot water', 'block driveway', 'nois street sidewalk', 'street condit', 'illeg park', 'street light condit', 'electron wast appoint', 'derelict vehicl', 'nois commerci', 'sanit condit', 'unsanitari condit', 'nois', 'abandon vehicl', 'sidewalk condit', 'graffiti', 'traffic signal condit', 'nois vehicl', 'derelict vehicl', 'new tree request', 'gener construct plumb', 'rodent', 'miss collect all materi', 'paint plaster']


In [55]:
# Assuming `comps_stem` is a list of stemmed text strings
comps_stem_no_stopwords = []
for comp_stem in comps_stem:
    words = comp_stem.split()
    words_no_stopwords = [word for word in words if word.lower() not in stop_words]
    comp_stem_no_stopwords = ' '.join(words_no_stopwords)
    comps_stem_no_stopwords.append(comp_stem_no_stopwords)

In [56]:
print(comps_stem_no_stopwords[0:25])

['request larg bulki item collect', 'nois residenti', 'heat hot water', 'block driveway', 'nois street sidewalk', 'street condit', 'illeg park', 'street light condit', 'electron wast appoint', 'derelict vehicl', 'nois commerci', 'sanit condit', 'unsanitari condit', 'nois', 'abandon vehicl', 'sidewalk condit', 'graffiti', 'traffic signal condit', 'nois vehicl', 'derelict vehicl', 'new tree request', 'gener construct plumb', 'rodent', 'miss collect materi', 'paint plaster']


In [57]:
# loop through each complaint in the list
for i in range(len(comps_stem_no_stopwords)):
    complaint = comps_stem_no_stopwords[i]
    # loop through each key in the dictionary
    for key in comp_dict_stem.keys():
        # if the complaint is in the list of values for this key
        if complaint in comp_dict_stem[key]:
            # replace the complaint with the key
            comps_stem_no_stopwords[i] = key
            # break out of the inner loop since we found a match
            break

In [58]:
print(len(comps_stem_no_stopwords))
comps_stem_no_stopwords = list(set(comps_stem_no_stopwords))
print(len(comps_stem_no_stopwords))

247
222


In [59]:
#process complaints and raw complaints
complaints_temp = []
for item in raw_complaints:
    item = np.char.replace(item, '/', ' ')
    symbols = "!\"\'#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        item = np.char.replace(item, i, '')
    complaints_temp.append(str(item))


rawcomps_stem = []
for s in range(len(complaints_temp)):
    x = stemSentence(complaints_temp[s])
    rawcomps_stem.append(x)
rawcomps_stem = [elem.strip() for elem in rawcomps_stem]
print(rawcomps_stem[0:25])

['park ticket lookup ticket or plate number known', 'servic request statu gener', 'properti tax account assist and bill inform', 'bulk item dispos', 'heat or hot water complaint in a residenti build', 'nois from neighbor', 'block driveway', 'park ticket or camera violat statu', 'servic request statu', 'apart mainten complaint', 'vehicl block driveway complaint', 'find a tow vehicl plate number known', 'bulk item pick up appoint', 'birth certif order', 'miss wast , wast , or organ collect', 'citi worker or agenc complaint or compliment', 'heat or hot water complaint in entir residenti build', 'illeg park', 'tow vehicl', 'divis transfer', 'illeg park complaint', 'bulk item dispos inform', 'cfc and freon remov', 'bu and subway', 'heat or hot water complaint in apart']


In [60]:
rawcomps_stem_no_stopwords = []
for comp_stem in rawcomps_stem:
    words = comp_stem.split()
    words_no_stopwords = [word for word in words if word.lower() not in stop_words]
    comp_stem_no_stopwords = ' '.join(words_no_stopwords)
    rawcomps_stem_no_stopwords.append(comp_stem_no_stopwords)
print(rawcomps_stem_no_stopwords[0:25])

['park ticket lookup ticket plate number known', 'servic request statu gener', 'properti tax account assist bill inform', 'bulk item dispos', 'heat hot water complaint residenti build', 'nois neighbor', 'block driveway', 'park ticket camera violat statu', 'servic request statu', 'apart mainten complaint', 'vehicl block driveway complaint', 'find tow vehicl plate number known', 'bulk item pick appoint', 'birth certif order', 'miss wast , wast , organ collect', 'citi worker agenc complaint compliment', 'heat hot water complaint entir residenti build', 'illeg park', 'tow vehicl', 'divis transfer', 'illeg park complaint', 'bulk item dispos inform', 'cfc freon remov', 'bu subway', 'heat hot water complaint apart']


In [61]:
# for i in range(len(raw_complaints)):
#     for key, values in comp_dict_final.items():
#         if any(value in raw_complaints[i] for value in values):
#             raw_complaints[i] = key
#             break

In [62]:
# print(len(raw_complaints))
# raw_complaints = list(set(raw_complaints))
# print(len(raw_complaints))

In [63]:
#try with the stemmed and cleaned complaints
for i in range(len(rawcomps_stem_no_stopwords)):
    for key, values in comp_dict_stem.items():
        if any(value in rawcomps_stem_no_stopwords[i] for value in values):
            rawcomps_stem_no_stopwords[i] = key
            break

In [64]:
print(len(rawcomps_stem_no_stopwords))
rawcomps_stem = list(set(rawcomps_stem_no_stopwords))
print(len(rawcomps_stem_no_stopwords))

5423
5423


In [65]:
import re

# result_dict = {}
# for comp in complaints:
#     for raw_comp in raw_complaints:
#         if re.search(r'\b{}\b'.format(comp), raw_comp):
#             if comp in result_dict:
#                 result_dict[comp].append(raw_comp)
#             else:
#                 result_dict[comp] = [raw_comp]
# result_dict

In [66]:
# result_dict_stem = {}
# for comp in comps_stem:
#     for raw_comp in rawcomps_stem:
#         if re.search(r'\b{}\b'.format(comp), raw_comp):
#             if comp in result_dict_stem:
#                 result_dict_stem[comp].append(raw_comp)
#             else:
#                 result_dict_stem[comp] = [raw_comp]
# result_dict_stem

In [67]:
# result_dict_stem_no_stopwords = {}

# for comp in comps_stem_no_stopwords:
#     comp_words = comp.split()  # Split the comp text into individual words
#     for raw_comp in rawcomps_stem_no_stopwords:
#         # Check if at least one word in comp_words is in the raw_comp string
#         if any(re.search(r'\b{}\b'.format(comp_word), raw_comp) for comp_word in comp_words):
#             if comp in result_dict_stem_no_stopwords:
#                 result_dict_stem_no_stopwords[comp].append(raw_comp)
#             else:
#                 result_dict_stem_no_stopwords[comp] = [raw_comp]
                
# result_dict_stem_no_stopwords

In [68]:
print(comps_stem_no_stopwords)

['mobil food vendor', 'illeg park', 'disorderli youth', 'hous option', 'public payphon complaint', 'consum complaint', 'overgrown tree branch', 'illeg firework', 'advoc', 'singl occup bathroom', 'vacant lot', 'sidewalk condit', 'cool tower', 'floor stair', 'anim abus', 'plumb', 'squeege', 'dpr intern', 'hous low incom senior', 'food poison', 'dirti condit', 'advocateproperti valu', 'special oper', 'rodent', 'bottl water', 'electron wast appoint', 'smoke', 'taxpay advoc inquiri', 'tan', 'damag tree', 'dof park tax exempt', 'drie', 'nois', 'question', 'paint plaster', 'hazard materi', 'radioact materi', 'lifeguard', 'dof properti payment issu', 'facade', 'bridg condit', 'peel paint', 'nonresidenti heat', 'industri wast', 'sweep missedinadequ', 'poison ivi', 'calori label', 'street sidewalk road condit', 'construct safeti enforc', 'mosquito', 'unsanitari condit', 'anim facil permit', 'stand water', 'dof properti rpie issu', 'alzheim care', 'home care provid complaint', 'build marshal offi

In [69]:
len(comps_stem_no_stopwords)

222

In [70]:
len(complaints)

247

In [ ]:
result_dict_stem_no_stopwords = {}

for comp_value in comps_stem_no_stopwords:
    if len(comp_value.split()) == 1: # if the comp_value has only one word
        for raw_comp in rawcomps_stem_no_stopwords:
            if re.search(r'\b{}\b'.format(comp_value), raw_comp):
                if comp_value in result_dict_stem_no_stopwords:
                    result_dict_stem_no_stopwords[comp_value].append(raw_comp)
                else:
                    result_dict_stem_no_stopwords[comp_value] = [raw_comp]
    else: # if the comp_value has more than one word
        comp_words = comp_value.split()
        for raw_comp in rawcomps_stem_no_stopwords:
            raw_comp_words = raw_comp.split()
            if sum([1 for w in comp_words if w in raw_comp_words]) >= 2:
                if comp_value in result_dict_stem_no_stopwords:
                    result_dict_stem_no_stopwords[comp_value].append(raw_comp)
                else:
                    result_dict_stem_no_stopwords[comp_value] = [raw_comp]

In [ ]:
result_dict_stem_no_stopwords

In [ ]:
unassigned_complaints = [raw_comp for raw_comp in raw_complaints if all(raw_comp not in values for values in result_dict.values())]

print(len(unassigned_complaints))

In [ ]:
# Find elements in raw_complaints that were not assigned as values in result_dict
unassigned_complaints_stem = [raw_comp for raw_comp in rawcomps_stem if all(raw_comp not in values for values in result_dict_stem.values())]

print(len(unassigned_complaints_stem))

In [ ]:
unassigned_complaints

In [ ]:
unassigned_complaints_stem

In [ ]:
values_to_remove_drinking = ['request lead test kit for drinking water','building drinking water tank inspections - reporting',
                    'illness caused by drinking water','drinking water','lead in drinking water',
                    'reduce risk of lead in drinking water','drinking water information']
values_to_remove_plant = ['wastewater treatment plant complaint - priority','wastewater treatment plant complaint']
# Remove the values for key 'drinking'
result_dict['drinking'] = [value for value in result_dict['drinking'] if value not in values_to_remove_drinking]
result_dict['plant'] = [value for value in result_dict['plant'] if value not in values_to_remove_plant]
# Add the values for key 'sewer'
result_dict['sewer'].extend(values_to_remove_plant)

In [ ]:
len(list(result_dict.values()))